In [2]:
cd drive/My Drive/chi_phi_y_te


[Errno 2] No such file or directory: 'drive/My Drive/chi_phi_y_te'
/content/drive/My Drive/chi_phi_y_te


In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [4]:
dataframe = pd.read_csv("diabetes.csv")
dataframe.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [5]:
train, test = train_test_split(dataframe, test_size = 0.2)
train, val = train_test_split(train, test_size = 0.2)
print(len(train))
print(len(val))
print(len(test))

491
123
154


In [0]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('Outcome')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [7]:
dataframe.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

In [0]:
train_outcome = train.pop('Outcome')


In [0]:
dataset_train = tf.data.Dataset.from_tensor_slices((train.values, train_outcome.values))

In [0]:
val_outcome = val.pop('Outcome')
dataset_val = tf.data.Dataset.from_tensor_slices((val.values, val_outcome.values))

In [0]:
test_outcome = test.pop('Outcome')
dataset_test = tf.data.Dataset.from_tensor_slices((test.values, test_outcome.values))

In [11]:
for feat, outcome in dataset_train.take(10):
  print('Features: {}, Outcome: {}'.format(feat, outcome))

Features: [  3.    182.     74.      0.      0.     30.5     0.345  29.   ], Outcome: 1
Features: [  4.    145.     82.     18.      0.     32.5     0.235  70.   ], Outcome: 1
Features: [  2.    124.     68.     28.    205.     32.9     0.875  30.   ], Outcome: 1
Features: [7.00e+00 1.42e+02 9.00e+01 2.40e+01 4.80e+02 3.04e+01 1.28e-01 4.30e+01], Outcome: 1
Features: [  8.    110.     76.      0.      0.     27.8     0.237  58.   ], Outcome: 0
Features: [  1.    100.     74.     12.     46.     19.5     0.149  28.   ], Outcome: 0
Features: [ 6.     0.    68.    41.     0.    39.     0.727 41.   ], Outcome: 1
Features: [ 11.    120.     80.     37.    150.     42.3     0.785  48.   ], Outcome: 1
Features: [  4.   189.   110.    31.     0.    28.5    0.68  37.  ], Outcome: 0
Features: [  6.    119.     50.     22.    176.     27.1     1.318  33.   ], Outcome: 1


In [0]:
train_dataset = dataset_train.shuffle(len(train)).batch(1)

In [0]:
val_dataset = dataset_val.shuffle(len(val)).batch(1)

In [0]:
def get_compiled_model():
  model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                metrics=['accuracy'])
  return model

In [16]:
model = get_compiled_model()
model.fit(train_dataset, 
          validation_data = val_dataset,
          epochs=15)

Epoch 1/15

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

491/491 [==============================] - 1s 3ms/step - loss: 1.8387 - accuracy: 0.6069 - val_loss: 1.8967 - val_accuracy: 0.5528
Epoch 2/15
491/491 [==============================] - 1s 3ms/step - loss: 1.3164 - accuracy: 0.6558 - val_loss: 1.3251 - val_accuracy: 0.6098
Epoch 3/15
491/491 [==============================] - 1s 3ms/step - loss: 0.7982 - accuracy: 0.6680 - val_loss: 0.6019 - val_accuracy: 0.7154
Epoch 4/15
491/491 [==============================] - 1s 3ms/step - loss: 0.7473 -

In [0]:
model.save('diabetes.h5')

In [0]:
test_dataset = dataset_test.shuffle(len(test)).batch(1)

In [23]:
loss, accuracy = model.evaluate(test_dataset)

154/154 [==============================] - 0s 2ms/step - loss: 0.5764 - accuracy: 0.7078


In [24]:
print("accuracy: ", accuracy)

accuracy:  0.7077922224998474
